In [38]:
from prefect import task, flow
import pandas as pd
import requests
import sqlite3

# Definir as tarefas do pipeline
@task
def extract_data():
    api_restcountries = "https://restcountries.com/v3.1/all"
    response = requests.get(api_restcountries)

    if response.status_code == 200:
        print("Importação com sucesso!")
        data = response.json()

        df = pd.json_normalize(data)
        return df
    else:
        print("Falha na importação!")
        return pd.DataFrame() 

@task
def transform_data(df):
    if not df.empty:
        # Selecionar colunas específicas
        df_paises = df[['name.common', 'capital', 'population', 'continents']].copy()

        # Aplicar transformações nas colunas
        df_paises['capital'] = df_paises['capital'].apply(lambda x: x[0] if isinstance(x, list) else x)
        df_paises['capital'] = df_paises['capital'].fillna('Sem Capital')
        df_paises['population'] = df_paises['population'].fillna(0)
        df_paises['continents'] = df_paises['continents'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
        df_paises['name.common'] = df_paises['name.common'].apply(lambda x: x if isinstance(x, str) else 'Desconhecido')

        # Renomear as colunas
        df_paises = df_paises.rename(columns={'name.common': 'Nome do Pais', 'capital': 'Capital', 'population': 'Populacao', 'continents': 'Continentes'})

        # Filtrar países com população superior a 1 milhão
        df_paises = df_paises[df_paises['Populacao'] > 1_000_000]
        print("Dados transformados com sucesso!")
        return df_paises
    else:
        print("Erro: DataFrame vazio na transformação.")
        return pd.DataFrame()

@task
def load_data(df):
    if not df.empty:
        con = sqlite3.connect('paises.db')
        cur = con.cursor()

        cur.execute("DROP TABLE IF EXISTS paises")
        con.commit()

        cur.execute('''
        CREATE TABLE IF NOT EXISTS paises (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            Nome_do_Pais TEXT,
            Capital TEXT,
            Populacao INTEGER,
            Continentes TEXT
        )
        ''')
        con.commit()

        # Inserir dados no banco
        for index, row in df.iterrows():
            cur.execute('''
            INSERT INTO paises (Nome_do_Pais, Capital, Populacao, Continentes)
            VALUES (?, ?, ?, ?)''', (row['Nome do Pais'], row['Capital'], row['Populacao'], row['Continentes']))
        
        con.commit()
        con.close()
        print("Dados inseridos com sucesso no banco de dados!")
    else:
        print("Erro: DataFrame vazio na carga de dados.")

# Definir o Flow do Prefect para orquestrar as tarefas
@flow
def etl_pipeline():
    data = extract_data()
    cleaned_data = transform_data(data)
    load_data(cleaned_data)



In [39]:
# Executar o fluxo manualmente para verificar se funciona corretamente
etl_pipeline()

20:09:24.793 | INFO    | prefect.engine - Created flow run 'poised-beetle' for flow 'etl-pipeline'

20:09:25.877 | INFO    | Task run 'extract_data-0ba' - Created task run 'extract_data-0ba' for task 'extract_data'

Importação com sucesso!


20:09:27.815 | INFO    | Task run 'extract_data-0ba' - Finished in state Completed()

20:09:28.899 | INFO    | Task run 'transform_data-c05' - Created task run 'transform_data-c05' for task 'transform_data'

...

20:09:29.166 | INFO    | Task run 'transform_data-c05' - Finished in state Completed()

20:09:30.416 | INFO    | Task run 'load_data-1d3' - Created task run 'load_data-1d3' for task 'load_data'

Dados inseridos com sucesso no banco de dados!


20:09:30.494 | INFO    | Task run 'load_data-1d3' - Finished in state Completed()

20:09:30.795 | INFO    | Flow run 'poised-beetle' - Finished in state Completed()